<a href="https://colab.research.google.com/github/Zuyuf/Machine_Learning_Specialization/blob/main/2%20-%20Machine%20Learning%20-%20Regression/Week%205%20-%20Feature%20Selection%20%26%20Lasso/2_Feature_Selection_and_LASSO_(Interpretation)_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install turicreate

     |████████████████████████████████| 91.9MB 101kB/s 
     |████████████████████████████████| 2.1MB 47.0MB/s 
     |████████████████████████████████| 86.3MB 46kB/s 
     |████████████████████████████████| 20.2MB 1.3MB/s 
     |████████████████████████████████| 3.4MB 46.1MB/s 
     |████████████████████████████████| 3.8MB 44.2MB/s 
     |████████████████████████████████| 450kB 45.5MB/s 
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0
  Found existing installation: google-auth-oauthlib 0.4.0
    Uninstalling google-auth-oauthlib-0.4.0:
      Successfully uninstalled google-auth-oauthlib-0.4.0
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tenso

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [ ]:
import turicreate
import numpy as np
import math

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
resource_dir = '/content/gdrive/My Drive/Turicreate/2 - ML - Regression/WEEK 5 - Feature Selection & Lasso/0 - Resources/'
sales = turicreate.SFrame(resource_dir + 'home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [ ]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [ ]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [ ]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.659810     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.877357     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [ ]:
non_zero_weights = model_all.coefficients[ model_all.coefficients['value'] > 0 ]
non_zero_weights

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [ ]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [ ]:
import pprint
validation_RSS = {}

for l1_penalty in np.logspace(1, 7, num=13):
    model = turicreate.linear_regression.create( training, target='price', features=all_features, 
                                                  l2_penalty=0, l1_penalty=l1_penalty, 
                                                  validation_set=None, verbose=False)
    
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    RSS = sum( residuals ** 2 )

    validation_RSS[l1_penalty] = RSS

pprint.pprint(validation_RSS)
print min(validation_RSS.items(), key= lambda x: x[1])

{10.0: 625766285142461.4,
 31.622776601683793: 625766285362395.2,
 100.0: 625766286057886.9,
 316.22776601683796: 625766288257224.9,
 1000.0: 625766295212185.9,
 3162.2776601683795: 625766317206077.8,
 10000.0: 625766386760661.5,
 31622.776601683792: 625766606749281.4,
 100000.0: 625767302791633.4,
 316227.7660168379: 625769507643885.0,
 1000000.0: 625776517727025.9,
 3162277.6601683795: 625799062845466.9,
 10000000.0: 625883719085424.5}
(10.0, 625766285142461.4)


### **QUIZ QUESTION**
- What was the best value for the `l1_penalty`?

In [ ]:
best_l1_penalty = min(validation_RSS.items(), key= lambda x: x[1])
best_l1_penalty

(10.0, 625766285142461.4)

- What is the RSS on TEST data of the model with the best `l1_penalty`?

In [ ]:
best_l1_model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                      l2_penalty=0, l1_penalty=best_l1_penalty[0],
                                                      validation_set=None, verbose=False)

predictions_test = best_l1_model.predict(testing)
residuals_test = testing['price'] - predictions_test
RSS_test = sum( residuals_test ** 2 )

print RSS_test

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [ ]:
best_l1_weights = best_l1_model.coefficients[best_l1_model.coefficients['value'] > 0]
best_l1_weights.print_rows(num_rows=18)
len(best_l1_weights)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [ ]:
store_NNZ = {}

for l1_penalty in l1_penalty_values:
  model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, verbose=False,
                                                l2_penalty=0, l1_penalty=l1_penalty)
  
  store_NNZ[l1_penalty] = model.coefficients['value'].nnz()

pprint.pprint(store_NNZ)

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 2976351441.6313133     # 10 non-zero weights
l1_penalty_max = 3792690190.7322536     # 6 non-zero weights

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values_linespace = np.linspace(l1_penalty_min, l1_penalty_max, 20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [ ]:
validation_RSS = {}

for l1_penalty in l1_penalty_values_linespace:
  model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                l2_penalty=0, l1_penalty=l1_penalty,
                                                validation_set=None, verbose=False)
  
  predictions = model.predict(validation)
  residuals = validation['price'] - predictions
  RSS = sum(residuals ** 2)

  validation_RSS[l1_penalty] = (RSS, model.coefficients['value'].nnz())

In [ ]:
best_RSS = float('inf')
best_l1 = None
for k, v in validation_RSS.iteritems():
  if (v[1] == max_nonzeros) and (v[0] < best_RSS):
    best_RSS = v[0]
    best_l1 = k

print best_l1, best_RSS

3448968612.163437 1.04693748875e+15


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [ ]:
model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=best_l1)

In [ ]:
non_zero_weight_test = model.coefficients[ model.coefficients["value"] > 0 ]
non_zero_weight_test.print_rows(num_rows=8)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]

